# Import and Map Business Terms to Data Headers

## Introduction
This notebook imports a business glossary for the industry accelerator into Watson Knowledge Catalog (WKC) for use in governing data assets. It also connects the data assets used in the accelerator to the WKC business terms.

In the first part of the notebook category and business terms are imported and then business terms are published into Watson Knowledge Catalog. The category and business terms csvs files are included with the project.

In the second part of the notebook we programmatically publish a dataset into a catalog and map business terms to the dataset column headers. The business terms and their mappings are specified in a csv file included with the project. The user must first ensure that the catalog exists and the imported business terms have been published.

The user can also assign business terms to column headers manually or by using the Metadata Enrichment capability within Cloud Pak for Data. 

This notebook is optional. The analytics project runs as expected even if this notebook is not used. 


#### Create Catalog

The catalog must be manually created. Under **Catalogs** in the navigation menu, select **All Catalogs** and select **New Catalog**. Enter the name for the catalog and the description if necessary and create the catalog. If the user has already created the catalog this step can be skipped and the existing catalog name should be specified in the code cell below.

 **catalog_name :** Name of the catalog that we would like to publish the csv to. This catalog is created based on the instructions above or an existing catalog.

In [1]:
catalog_name = 'Industry Accelerators'

In [2]:
# imports for the rest APIs interactions with WKC
import requests
from requests.packages.urllib3.exceptions import InsecureRequestWarning
import json
requests.packages.urllib3.disable_warnings(InsecureRequestWarning)
from pandas.io.json import json_normalize
import pandas as pd
import os
import time 

s=requests.session()

from ibm_watson_studio_lib import access_project_or_space
wslib = access_project_or_space()

## User Inputs
1. **wkcURL**: The base url used to call the apis of data platform.
2. **categories_csv**: Name of the csv file containing the categories.
3. **terms_csv**: Name of the csv file containing the business terms and their definitions.

The user does not need to change the code cell below, unless they run this notebook on a different environment or changed the name of the csv file with categories and business terms.


In [3]:
wkcURL = os.environ['RUNTIME_ENV_APSX_URL']
categories_csv="customer-attrition-prediction-glossary-categories.csv"
terms_csv="customer-attrition-prediction-glossary-terms.csv"

We also create additional variables. The user does not need to change the code cell below, unless they change the business terms category name or the name of the csv file with mappings.

1. **category_name :** Name of the business term category corresponding to the project.
2. **terms_file :** Name of the csv file containing the list of mappings between column headers and business terms.
3. **data_asset_file_to_publish :** Name of the csv files that will be published into the catalog and for which we map business terms.

In [4]:
category_name = "Customer Attrition Prediction"
terms_file = "customer-attrition-map-terms.csv" 
data_asset_file_to_publish = ["customer_history.csv", "customer_summary.csv", "customer.csv", "account.csv"]

## 1. Import the Categories and Business terms 
If the user is successfully authenticated, then we complete the following steps 
1. Read the business categories file and import the category `Customer Attrition Prediction` under `Industry Accelerators`
2. Read the business terms file and import the business terms with definitions and related/part of terms for `Customer Attrition Prediction`.
3. The imported business terms will be saved as a draft, publish the business terms programmatically. 

#### Import the Category

Below cell reads catgories csv file and import the category `Customer Attrition Prediction` under `Industry Accelerators`.  

In [5]:

wslib.download_file(categories_csv)

# Create a header
importheaders = {
    'accept': 'application/json',
    'Authorization': 'Bearer '+os.environ['USER_ACCESS_TOKEN']  
    #'Content-Type': 'multipart/form-data'
}

# Specify merge option 
params = (
    ('merge_option', 'all'),
)

# remove the index if necessary
time.sleep(10)

# Specify the file name to import
files = {
    'file': (categories_csv, open(categories_csv, 'rb')),
}

try:
    categoryimport_url = wkcURL+'/v3/governance_artifact_types/category/import'
    cat_import = s.post(categoryimport_url, headers=importheaders, params=params, files=files)
    if cat_import.status_code in (200,202):
        print("Category import has",json.loads(cat_import.text)['status'])
    else:
        print("Category import is unsuccessful,", json.loads(cat_import.text)['errors'][0]['message'])
except:
    print("The below error has occurred. Please ensure that categories csv file exists")
    raise



Category import has SUCCEEDED


#### Import Business terms

Read the business terms file and import the business terms with definitions and related/part of terms for `Customer Attrition Prediction`. `Lite` plan users can import 300 business terms into watson knowledge catalog. If the user already has more business terms, delete them before running below cell. 


In [6]:

wslib.download_file(terms_csv)



files = {
    'file': (terms_csv, open(terms_csv, 'rb')),
}

try:
    termsimport_url =wkcURL+'/v3/governance_artifact_types/glossary_term/import'
    term_import = s.post(termsimport_url, headers=importheaders, params=params, files=files)
    if term_import.status_code in (200,202):
        wf_json=json.loads(term_import.text)
        print("Terms import has",wf_json['status'])
        print("Number of Business terms imported:",max(json.loads(term_import.text)['operations_count']['glossary_term']['IMPORT_MODIFY'],json.loads(term_import.text)['operations_count']['glossary_term']['IMPORT_CREATE']))
    else:
        print("Import of Business Terms failed.")
        raise Exception(json.loads(term_import.text)['errors'][0]['message'])
except:
    print("The below error has occurred. Please check your WKC plan and Business Terms csvs")
    print(json.loads(term_import.text)['messages']['resources'])
    raise



Terms import has SUCCEEDED
Number of Business terms imported: 151


#### Fetch imported terms draft and publish the terms

If the business terms are successfully imported in the previous cell, the imported business terms have been saved in draft status. The below cell publishes the business terms.

In [7]:
headers = {
'Content-Type': "application/json",
'Authorization': "Bearer "+os.environ['USER_ACCESS_TOKEN']

}
try:
    wf_url=wkcURL+"/v3/workflows/"+wf_json['workflow_id']+"?includeUserTasks=true"
    wf_response=s.get(wf_url,headers=headers)
    if wf_response.status_code in (200,202):
        idtopublish=json.loads(wf_response.text)['entity']['user_tasks'][0]['metadata']['task_id']
except:
    print("The below error has occurred. Please ensure that terms are imported correctly")
    raise    
    
    
## Publish the Business terms
data = {
    "action": "complete",
    "form_properties": [
        {
            "id": "action",
            "value": "#publish"
        }
    ]
}
publish_url=wkcURL+'/v3/workflow_user_tasks/'+idtopublish+'/actions'


try:
    publish = s.post(publish_url, headers=headers, json=data, verify=False)

    if publish.status_code in (200,202,204):
        print("The Business terms are published.")
except:
    print("The below error has occurred. Please ensure that terms are imported correctly")
    raise
    


The Business terms are published.


When the Business terms are published, navigate to **Governance -> Categories -> Industry Accelerators** to find the category and business terms pertaining to this accelerator.


## 2. Map Business Terms to Data Headers
In this part of the notebook, we take the published business terms and map them to the dataset column headers



## Map Business Terms to Headers

We complete the following steps to map the business terms to column headers:

1. Check if the Category `Customer Attrition Prediction` exists in the parent category `Industry Accelerators`.
2. Load the business terms from the `Customer Attrition Prediction` subcategory into a dataframe.
3. Publish the specified dataset into the catalog.
4. Assign business terms to the dataset column headers.

### 1. Check for the Category

In [8]:
search_url=wkcURL+"/v3/search"
try:
    headers = {
        'Content-Type': "application/json",
        'Authorization': "Bearer "+os.environ['USER_ACCESS_TOKEN'],
        'Cache-Control': "no-cache",
        'Connection': "keep-alive"
        }
    
    search_body = {
        "size": 1000,
        "_source": ["artifact_id","metadata.name"],
       "query": {    
               "match": {"metadata.artifact_type": "category"}
       }
    }
    parent_cat = s.post(search_url, verify=False,  json=search_body, headers=headers)
    
    
    
    # Check if Industry accelerator category exists and load its id into a variable `parent_id`
    if parent_cat.status_code == 200:
        category=json.loads(parent_cat.text)
        for i in category['rows']:
            
            if i['metadata']['name']== category_name:
                print("Category ",category_name,"exists")

                exists_category=True
                category_id=i['artifact_id'] 
                #category_id=cat_id[cat_id.index('_')+1:][:cat_id.index('_')]
            
                   
except:
    print("The below error has occurred. " + "Please ensure that category, '" + category_name + "', exists.")
    raise ValueError(parent_cat.text)

Category  Customer Attrition Prediction exists


### 2. Load category Business Terms into Dataframe 

Get all of the terms in the `Customer Attrition Prediction` category and store them in the `df_terms` dataframe.

In [9]:

time.sleep(25)
# Create a payload for the post request, This payload contains information on size of the terms, source, category and subcategory ids
payload={"size":300,"from":0,"_source":["artifact_id","metadata.artifact_type","metadata.name","metadata.description","categories","entity.artifacts"],"query":{"bool":{"filter":{"bool":{"minimum_should_match":1,"should":[{"term":{"categories.primary_category_id":category_id}},{"term":{"categories.secondary_category_ids":category_id}}],"must_not":{"terms":{"metadata.artifact_type":["category"]}}}}}}}
# create a post request with above payload 
wf=s.post(wkcURL+"/v3/search",headers=headers,json=payload,verify=False)
# it will return all the terms , load these terms into a dataframe
wf_json=json.loads(wf.text)['rows']
df_terms=pd.json_normalize(wf_json)

df_terms=df_terms[['entity.artifacts.global_id','metadata.name']]

In [10]:
# terms dataframe looks as below
df_terms.head()

,entity.artifacts.global_id,metadata.name
0,a13155a7-93d4-4de9-a9e9-4e7c84fa2b0a_fe0c17af-...,City
1,a13155a7-93d4-4de9-a9e9-4e7c84fa2b0a_b124ec60-...,Work Address
2,a13155a7-93d4-4de9-a9e9-4e7c84fa2b0a_07785e2d-...,Customer Account Deposits
3,a13155a7-93d4-4de9-a9e9-4e7c84fa2b0a_b9997a6a-...,Customer Tenure
4,a13155a7-93d4-4de9-a9e9-4e7c84fa2b0a_6a6e243b-...,Customer Account Commision Amount


### 3. Publish Dataset into Catalog

Get the ID of the catalog that was specified in the user inputs at the beginning of this notebook.

In [11]:
## Get catalog that created and its id by providing name of the catalog created, wich should be same as the one entered in the previous cells
catalog_endpoint=wkcURL+"/v2/catalogs"
# Create new header for the requests
headers = {
'Content-Type': "application/json",
'Authorization': "Bearer "+os.environ['USER_ACCESS_TOKEN']

}

# endpoint to get all the catalogs 
get_catalog=s.get(catalog_endpoint,verify=False, headers=headers)


## Find the catalog created with specific name and store name and id of it into catalog_name and catalog_id respectively
try:
    get_catalog_json=json.loads(get_catalog.text)['catalogs']
except:
    print("The below error has occurred. Please ensure that catalog, '" + catalog_name + "', exists")
    raise
    
catalog_id = ''
for metadata in get_catalog_json:
    if metadata['entity']['name']==catalog_name:
        catalog_id=metadata['metadata']['guid']
        print("catalog_id for",catalog_name, catalog_id)

if catalog_id == '':
    print("The provided catalog name cannot be found. Please ensure that catalog, '" + catalog_name + "', exists")
    raise ValueError("Catalog cannot be found")

catalog_id for Industry Accelerators 73768cf9-0b37-45dd-8c9a-d6446d7377c0


Get the project id. All project assets can be accessed using this project id.

In [12]:
# Enter project_id manually if referring to a different project
project_id=os.environ['PROJECT_ID']

Get all existing csv files in the project folder and store the names of these files. 

In [13]:
# payload 
payload={"query":"*:*","limit":200}
# endpoint to access all the project assets in the project folder 
asset_url=wkcURL+"/v2/asset_types/asset/search?project_id="+project_id
get_asset=s.post(asset_url,json=payload,verify=False)

Next we get the asset id of the dataset to be published to the catalog.

In [14]:
# Get asset ids of all csv files to be published in to the catalog and store the asset ids in an array

project_asset_id=[]
# Payload to query all project assets
payload={"query":"*:*","limit":200}

get_asset=s.post(wkcURL+"/v2/asset_types/asset/search?project_id="+project_id,json=payload,verify=False, headers=headers)
get_asset_json=json.loads(get_asset.text)
for j in get_asset_json['results']:
    if j['metadata']['name'] in data_asset_file_to_publish:
        print("Asset id of",j['metadata']['name'],":",j['metadata']['asset_id'])
        project_asset_id.append(j['metadata']['asset_id'])

Asset id of customer.csv : 06311439-619e-42f9-8797-6e7297980968
Asset id of account.csv : 587b9479-d609-49ee-8150-d1d7802f52b0
Asset id of customer_history.csv : d855403d-dba7-4234-9c3c-daca1b7eacf1
Asset id of customer_summary.csv : bf2e9378-e912-4d91-816e-b3542a1a4c9f


Using the asset ID for the dataset, upload the dataset into the catalog using the post request below. Get the new asset ID of the newly published dataset.

In [15]:
print("ASSET ID's of the published assets")
# Creates a empty dictionary
catalog_asset_ids={}
for asset_id in project_asset_id:
    #for  each asset in the project , publish them into the catalog 
    # pyload to publish the asset
    payload={"mode":0,"catalog_id":catalog_id,"metadata":{}}
    # endpoint to publish asset
    asset_publish_url=wkcURL+"/v2/assets/"+asset_id+"/publish?project_id="+project_id
    # Post request with endpoint, heaeder and payload
    publishasset=requests.post(asset_publish_url,json=payload,headers=headers,verify=False)
    # api endpoint returns below text
    publishasset_json=json.loads(publishasset.text)
    # extract csv file published and its asset id and append it to the dictionary
    catalog_asset_ids[publishasset_json['metadata']['name']]=publishasset_json['asset_id']
    
print(catalog_asset_ids)

ASSET ID's of the published assets
{'customer.csv': 'a050723c-7629-4316-a779-89dcd13d6a11', 'account.csv': 'd365122c-acfd-44f0-998e-de926add2c41', 'customer_history.csv': '996f9262-c065-49b6-bc0e-db0005767755', 'customer_summary.csv': '7091fafc-62fc-4a8c-b650-8e9ffda49d9c'}


### 4. Assign Business Terms to Column Headers

Read in the file with business terms and their associated column headers and view a sample of the data.

In [16]:
map_terms_file =wslib.load_data(terms_file)
map_terms = pd.read_csv(map_terms_file)
print(map_terms.shape)
map_terms.head()

(301, 4)


,Business Terms,Column_header,Table,File
0,Account Balance,OPENING_BALANCE,CUSTOMER-ATTRITION,ACCOUNT_SUMMARY
1,Account Balance,CUSTOMER_SUMMARY_TOTAL_CLOSING_CASH_BALANCE,CUSTOMER-ATTRITION,CUSTOMER_HISTORY
2,Account Balance,CUSTOMER_SUMMARY_TOTAL_CLOSING_BALANCE_LOANS,CUSTOMER-ATTRITION,CUSTOMER_HISTORY
3,Account Balance,OPENING_CASH_BALANCE,CUSTOMER-ATTRITION,ACCOUNT_SUMMARY
4,Account Balance,CUSTOMER_SUMMARY_TOTAL_CLOSING_BALANCE,CUSTOMER-ATTRITION,CUSTOMER_HISTORY


In [17]:
df_terms.head()

,entity.artifacts.global_id,metadata.name
0,a13155a7-93d4-4de9-a9e9-4e7c84fa2b0a_fe0c17af-...,City
1,a13155a7-93d4-4de9-a9e9-4e7c84fa2b0a_b124ec60-...,Work Address
2,a13155a7-93d4-4de9-a9e9-4e7c84fa2b0a_07785e2d-...,Customer Account Deposits
3,a13155a7-93d4-4de9-a9e9-4e7c84fa2b0a_b9997a6a-...,Customer Tenure
4,a13155a7-93d4-4de9-a9e9-4e7c84fa2b0a_6a6e243b-...,Customer Account Commision Amount


Join the `df_terms` and `map_terms` dataframes and map each column header to a business term. The code below loops through each file in the catalog (one file in our case) and performs the following tasks:

1. Create a dataframe with column headers in the catalog and associated business term and term ids.
2. Fetch catalog asset id for each csv in the catalog.
3. Create a column_info attribute for all the files in the catalog.
4. Map column header to the business terms. 

In [19]:
# For every file in the map terms csv do the following
# Join the csv with specified file name with the published terms to get its term id
# drop if any duplicates found to avoid multiple mappings for the same term

for catalog_file in catalog_asset_ids:
    # Catalog asset id of the particular csvs
    # for each file name in the map_terms if the csv with this file name exists, get its asset_id from the catalog and use the post request publish create column_info attribute
    # This column info attribute is necessary to map the busines terms to column to header
    
    file_name=catalog_file.replace(".csv","").upper()
    map_terms_file=map_terms[map_terms["File"]==file_name]
    Terms_Headers=pd.merge(map_terms_file,df_terms,left_on='Business Terms',right_on='metadata.name',how='inner')
    catalog_asset_id=catalog_asset_ids[catalog_file]
    print(catalog_file,  catalog_asset_id)
    #### 
    payload={"name": "column_info",
       "entity":{
                  #"sample_size":50
               }
    }
    t=requests.post(wkcURL+"/v2/assets/"+catalog_asset_id+"/attributes?catalog_id="+catalog_id,json=payload,headers=headers,verify=False)

    # For each column header in the file map its corresponding business term retrieved from the above join in the dataframe

    i=0
    for index, rows in Terms_Headers.iterrows(): 
        i+=1
        print(i,rows.Column_header.strip(), "is mapped to", rows['Business Terms'])
        # Create list for the current row 
        # Below payload is used for the patch request to map the  header to business terms
        payload=[{"op":"add","path":"/"+rows.Column_header.strip(),"value":{"column_terms":[{"term_display_name":rows['Business Terms'],"term_id":rows["entity.artifacts.global_id"]}]},"attribute":"column_info"}]
    #
        # Endpoint for patch request
        url=wkcURL+"/v2/assets/"+catalog_asset_id+"/attributes/column_info?catalog_id="+catalog_id
    # patch request to map busines terms to column header using term_id
        patch_attribute=s.patch(url,json=payload,headers=headers,verify=False)
    #
        json.loads(patch_attribute.text)

    

customer.csv a050723c-7629-4316-a779-89dcd13d6a11
1 DATE_FIRST_ACCOUNT_OPENED is mapped to Account Opened Date
2 DATE_LAST_ACCOUNT_OPENED is mapped to Account Opened Date
3 RELATIONSHIP_START_DATE is mapped to Account Opened Date
4 EFFECTIVE_DATE is mapped to Account Opened Date
5 RELATIONSHIP_START_DATE is mapped to Communication Start
6 DATE_FIRST_ACCOUNT_OPENED is mapped to Customer Account
7 DATE_LAST_ACCOUNT_OPENED is mapped to Customer Account
8 EFFECTIVE_DATE is mapped to Customer Account
9 ACQUISITION_COST is mapped to Customer Acquisition Cost
10 SPECIAL_TERMS_INDICATOR is mapped to Customer Agreement Type
11 RELATIONSHIP_START_DATE is mapped to Customer Agreement Validity Start
12 CUSTOMER_BEHAVIOR is mapped to Customer Behavior
13 CUSTOMER_ID is mapped to Customer Identifier
14 IMPORTANCE_LEVEL_CODE is mapped to Customer Importance
15 PRIMARY_WRITTEN_LANGUAGE is mapped to Customer Language
16 PRIMARY_SPOKEN_LANGUAGE is mapped to Customer Language
17 MARKET_GROUP is mapped to

60 CUSTOMER_SUMMARY_LATEST_SENTIMENT_SCORE is mapped to Customer Sentiment
61 CUSTOMER_SUMMARY_NUMBER_OF_POSTS is mapped to Customer Social Media Posts
62 CUSTOMER_STATUS is mapped to Customer Status
63 CUSTOMER_SUMMARY_TOTAL_AMOUNT_OF_TRANSACTION_FEES is mapped to Customer Transaction Fees Amount
64 CUSTOMER_SUMMARY_TOTAL_NUMBER_OF_DEPOSITS is mapped to Customer Units Bought
65 CUSTOMER_SUMMARY_TOTAL_NUMBER_OF_UNITS_BOUGHT is mapped to Customer Units Sold
66 CUSTOMER_SUMMARY_TOTAL_AMOUNT_OF_WAIVED_FEES is mapped to Customer Waived Fees Amount
67 CUSTOMER_SUMMARY_TOTAL_NUMBER_OF_WITHDRAWALS is mapped to Customer Withdrawals
68 CUSTOMER_SUMMARY_TOTAL_AMOUNT_OF_WITHDRAWALS is mapped to Customer Withdrawals Amount
69 CUSTOMER_BIRTH_YEAR is mapped to Date Of Birth
70 CUSTOMER_AGE_RANGE is mapped to Date Of Birth
71 CUSTOMER_EMPLOYMENT_STATUS is mapped to Employment Status
72 CUSTOMER_GENDER is mapped to Gender
73 NUM_ACCOUNTS_WITH_INVESTMENT_OBJECTIVE_GROWTH is mapped to Growth As Investme

48 TOTAL_NUMBER_OF_WITHDRAWALS is mapped to Customer Withdrawals
49 TOTAL_AMOUNT_OF_WITHDRAWALS is mapped to Customer Withdrawals Amount
50 TOTAL_INWARD_CALLS is mapped to Inbound Calls
51 TOTAL_INWARD_EMAILS is mapped to Inbound Communications
52 TOTAL_INWARD_COMMUNICATIONS is mapped to Inbound Communications
53 TOTAL_CLOSING_BALANCE_LOANS is mapped to Loan Account
54 NUMBER_OF_NEW_ACCOUNTS_OPENED is mapped to New Customer Agreements
55 TOTAL_NUMBER_OF_ACCOUNTS_AS_NON_PRIMARY is mapped to Number of Customer Accounts
56 TOTAL_NUMBER_OF_ACCOUNTS_AS_PRIMARY is mapped to Number of Customer Accounts
57 NUMBER_OF_ACCOUNTS is mapped to Number of Customer Accounts
58 NUMBER_OF_COMMUNICATIONS is mapped to Number of Customer Communications
59 TOTAL_OUTWARD_CALLS is mapped to Outbound Calls
60 TOTAL_OUTWARD_COMMUNICATIONS is mapped to Outbound Communications
61 TOTAL_OUTWARD_EMAILS is mapped to Outbound Communications
62 REAL_PROPERTY_ASSETS is mapped to Properties Owned
63 RETURN_5Y is mapped t

The specified dataset is now published to the catalog and its column headers are mapped to their associated business terms. 

Navigate to the business terms or the catalog data asset to view the mappings created.

The associated business term for the column header is displayed.

In [19]:
s.close()

<hr>

Sample Materials, provided under <a href="https://github.com/IBM/Industry-Accelerators/blob/master/CPD%20SaaS/LICENSE" target="_blank" rel="noopener noreferrer">license.</a> <br>
Licensed Materials - Property of IBM. <br>
© Copyright IBM Corp. 2020, 2023. All Rights Reserved. <br>
US Government Users Restricted Rights - Use, duplication or disclosure restricted by GSA ADP Schedule Contract with IBM Corp. <br>